In [20]:
import augmentor
import dataprovider3 as dp3
import h5py
import numpy as np
import os

### FIB-25 Validation Sample

In [21]:
home = os.path.expanduser('~')
fib25_path = os.path.join(home, 'Data_local/FIB-25/validation_sample')

In [22]:
fpath = os.path.join(fib25_path, 'img.h5')
with h5py.File(fpath, 'r') as f:
    img = f['/main'][...]

fpath = os.path.join(fib25_path, 'seg.h5')
with h5py.File(fpath, 'r') as f:
    seg = f['/main'][...]
    
fpath = os.path.join(fib25_path, 'msk.h5')
with h5py.File(fpath, 'r') as f:
    msk = f['/main'][...]

In [23]:
img = (img/255.).astype(np.float32)

In [24]:
# DataSet
dset = dp3.Dataset()
dset.add_data('img', img)
dset.add_data('seg', seg)
dset.add_mask('seg_mask', msk, loc=True)

In [25]:
# DataProvider
dims = (240,240,240)
spec = {'img': dims, 'seg': dims, 'seg_mask': dims}
dp = dp3.DataProvider(spec)
dp.add_dataset(dset)
dp.set_imgs(['img'])
dp.set_segs(['seg'])

In [26]:
sample = dp()

In [27]:
sample['img'].shape

(1, 240, 240, 240)

### 120nm Tilt Series

In [28]:
n = 12

In [29]:
projections = []
projections.append(augmentor.NormalView(n))
projections.append(augmentor.TiltedView(n,  1, -1))
projections.append(augmentor.TiltedView(n, -1, -1))
projections.append(augmentor.TiltedView(n,  1, -2))
projections.append(augmentor.TiltedView(n, -1, -2))

In [30]:
%time p0 = projections[0](sample['img'])

CPU times: user 7.41 ms, sys: 0 ns, total: 7.41 ms
Wall time: 7.18 ms


In [31]:
%time p1 = projections[1](sample['img'])

CPU times: user 24.8 ms, sys: 4.19 ms, total: 29 ms
Wall time: 28.1 ms


In [32]:
%time p2 = projections[2](sample['img'])

CPU times: user 25.3 ms, sys: 1.1 ms, total: 26.4 ms
Wall time: 25.5 ms


In [33]:
%time p3 = projections[3](sample['img'])

CPU times: user 19.8 ms, sys: 3.71 ms, total: 23.5 ms
Wall time: 22.5 ms


In [34]:
%time p4 = projections[4](sample['img'])

CPU times: user 22 ms, sys: 1.16 ms, total: 23.2 ms
Wall time: 22.3 ms


In [35]:
results = [p(sample['img']) for p in projections]

In [36]:
for r in results:
    print(r.shape)

(1, 20, 240, 240)
(1, 20, 240, 240)
(1, 20, 240, 240)
(1, 20, 240, 240)
(1, 20, 240, 240)


In [37]:
ts = np.concatenate(results, axis=0)

In [38]:
import napari

viewer = napari.view_image((ts*255).astype(np.uint8), name='image')
napari.run()